In [94]:
import gmsh
import pyvista as pv
import sys
import math
import numpy as np

In [ ]:
gmsh.initialize()
gmsh.model.add("model")

gmsh.option.setNumber("Geometry.SurfaceLabels", 1)
gmsh.option.setNumber("Geometry.CurveLabels", 1)  # Display surface's index
gmsh.option.setNumber("Geometry.PointLabels", 0)
# gmsh.option.setNumber("Mesh.RecombineAll", 1)  # Set mesh to quadrangle
# gmsh.option.setNumber("Mesh.MeshSizeMax", 5)
# gmsh.option.setNumber("Mesh.MeshSizeMin", 1)

# # Parameters from the input dictionary
# layers = [0.0008333347456249489, 0.0017800194633658768, 0.0028554719865078446, 0.004077207277949342,
#             0.005465122681218706, 0.007041821971258495, 0.008832983482512, 0.01086777830969492, 0.013179345392125584,
#             0.01580533121890038, 0.018788502944624648, 0.022177444900983473, 0.026027349847632435,
#             0.030400917848825662, 0.03536937741500555, 0.04101364553980197, 0.0474256455249578, 0.05470980405545308,
#             0.06298475190639223, 0.07238525597959321, 0.08306441413528344, 0.09519614856422366, 0.10897803830764989,
#             0.12463453705583005, 0.14242062863073102, 0.16262597968647283, 0.18557965725900274, 0.2116554879956303,
#             0.24127814634554928, 0.27493007086442706, 0.3131593212729187, 0.3565885042302057, 0.40592491318889573,
#             0.4619720474703085, 0.5256426981613417, 0.5979738139515741, 0.6801433890176068, 0.7734896479929753,
#             0.8795328404726032, 1]
# numElements = [1] * 40

# Create outer loop
# outerLoop = gmsh.model.occ.addRectangle(-250, -250, 0, 850, 500)  # Improve： Adjustable parameters (x_min, y_min, z, x_max, y_max)


stl_filename = "building.stl"
# Modified part: Extract bottom edges using the new method
mesh = pv.read(stl_filename)

# First extract all feature edges
all_feature_edges = mesh.extract_feature_edges(
    boundary_edges=True,
    feature_edges=True,
    manifold_edges=False,
    non_manifold_edges=True,
    feature_angle=20.0
)

In [96]:
# Get points and edges data
edge_points = all_feature_edges.points

# Convert lines to edge connectivity
line_cells = []
offset = 0
while offset < len(all_feature_edges.lines):
    n_points = all_feature_edges.lines[offset]
    line_cells.append(all_feature_edges.lines[offset + 1:offset + 1 + n_points])
    offset += n_points + 1

# Filter bottom edges
bottom_edges = []
for edge in line_cells:
    point1_z = edge_points[edge[0]][2]
    point2_z = edge_points[edge[1]][2]
    
    if np.isclose(point1_z, 0.0, atol=1e-6) and np.isclose(point2_z, 0.0, atol=1e-6):
        bottom_edges.append(edge)

In [97]:
# Create points using occ kernel
point_index_to_tag = {}
unique_points = set()

# Collect unique points from bottom edges
for edge in bottom_edges:
    for point_idx in edge:
        unique_points.add(point_idx)

# Create points in gmsh
for point_idx in unique_points:
    x, y, z = edge_points[point_idx]
    tag = gmsh.model.occ.addPoint(x, y, z)
    point_index_to_tag[point_idx] = tag
    
print('point_index_to_tag:', point_index_to_tag)


point_index_to_tag: {np.int64(0): 1, np.int64(3): 2, np.int64(6): 3, np.int64(7): 4, np.int64(8): 5, np.int64(12): 6, np.int64(13): 7, np.int64(14): 8, np.int64(17): 9, np.int64(18): 10, np.int64(21): 11, np.int64(23): 12, np.int64(25): 13, np.int64(26): 14, np.int64(29): 15, np.int64(31): 16, np.int64(32): 17, np.int64(34): 18, np.int64(37): 19, np.int64(38): 20, np.int64(41): 21, np.int64(43): 22, np.int64(44): 23, np.int64(46): 24, np.int64(49): 25, np.int64(50): 26, np.int64(51): 27, np.int64(54): 28, np.int64(57): 29, np.int64(58): 30, np.int64(61): 31, np.int64(62): 32, np.int64(65): 33, np.int64(68): 34, np.int64(70): 35, np.int64(72): 36, np.int64(74): 37, np.int64(76): 38, np.int64(79): 39}


In [98]:
# Create lines and store connectivity information
line_point_pairs = [] # (line_tag, start_point_tag, end_point_tag)
point_to_lines = {} # {point_tag: [line_tag1, line_tag2, ...]}

for edge in bottom_edges:
    start_point_tag = point_index_to_tag[edge[0]]
    end_point_tag = point_index_to_tag[edge[1]]
    
    line_tag = gmsh.model.occ.addLine(start_point_tag, end_point_tag)
    line_point_pairs.append((line_tag, start_point_tag, end_point_tag))
    
    for point_tag in [start_point_tag, end_point_tag]:
        if point_tag not in point_to_lines:
            point_to_lines[point_tag] = []
            
        point_to_lines[point_tag].append(line_tag) # Add line to point

In [99]:
unused_lines = set([line[0] for line in line_point_pairs])
curve_loops = []


## --------------- Previous --------------------#
# while unused_lines:
#     current_line_tag = unused_lines.pop()
#     for line_info in line_point_pairs:
#         if line_info[0] == current_line_tag:
#             break

#     start_point = line_info[1]
#     end_point = line_info[2]
#     ordered_lines = [line_info]

#     while end_point != start_point and unused_lines:  # Added check for unused_lines
#         connected_lines = point_to_lines[end_point]
#         next_line_tag = None
#         for l_tag in connected_lines:
#             if l_tag in unused_lines:
#                 next_line_tag = l_tag
#                 break
        
#         if next_line_tag is None:  # If no next line is found, break the loop
#             break

#         unused_lines.remove(next_line_tag)

#         for line_info in line_point_pairs:
#             if line_info[0] == next_line_tag:
#                 break

#         if line_info[1] == end_point:
#             new_end_point = line_info[2]
#         else:
#             new_end_point = line_info[1]
#             line_info = (-line_info[0], line_info[2], line_info[1])

#         ordered_lines.append(line_info)
#         end_point = new_end_point
#     gmsh.model.occ.synchronize()
#     curve_loop_lines = [line[0] for line in ordered_lines]
#     print("Current curve loop lines:", curve_loop_lines)
#     try:
#         curve_loop_tag = gmsh.model.occ.addCurveLoop(curve_loop_lines)
#         curve_loops.append(curve_loop_tag)
#     except:
#         print("Warning: Failed to create curve loop, skipping...")
#         continue

# ------------------ New -------------------#
line_usage_count = {}
for line in line_point_pairs:
    line_usage_count[abs(line[0])] = 0

while unused_lines:
    current_line_tag = unused_lines.pop() # 
    for line_info in line_point_pairs:
        if line_info[0] == current_line_tag:
            break
    else:
        print(f" {current_line_tag}")
        continue

    start_point = line_info[1]
    end_point = line_info[2]
    ordered_lines = [line_info]

    current_loop_lines = {abs(line_info[0])}
    line_usage_count[abs(line_info[0])] += 1

    while end_point != start_point and unused_lines:
        connected_lines = point_to_lines[end_point]
        next_line_tag = None
        
        # loop through all the line
        for l_tag in connected_lines:
            # check if the line can be used
            if l_tag in unused_lines and \
               (abs(l_tag) not in current_loop_lines) and \
               line_usage_count[abs(l_tag)] < 2:
                next_line_tag = l_tag
                break
        
        if next_line_tag is None:
            # find if any loop can be reuseable
            for l_tag in connected_lines:
                if line_usage_count[abs(l_tag)] < 2:
                    next_line_tag = l_tag
                    if l_tag in unused_lines:
                        unused_lines.remove(l_tag)
                    break
        
        if next_line_tag is None:
            print("can't find next line to create the loop")
            break

        if next_line_tag in unused_lines:
            unused_lines.remove(next_line_tag)

        for line_info in line_point_pairs:
            if line_info[0] == next_line_tag:
                break

        if line_info[1] == end_point:
            new_end_point = line_info[2]
        else:
            new_end_point = line_info[1]
            line_info = (-line_info[0], line_info[2], line_info[1])

        ordered_lines.append(line_info)
        current_loop_lines.add(abs(line_info[0]))
        line_usage_count[abs(line_info[0])] += 1
        end_point = new_end_point

    curve_loop_lines = [line[0] for line in ordered_lines]
    print("Current curve loop lines:", curve_loop_lines)
    
    try:
        curve_loop_tag = gmsh.model.occ.addCurveLoop(curve_loop_lines)
        curve_loops.append(curve_loop_tag)
        print(f"Sucessfully created {curve_loop_tag}")
    except Exception as e:
        print(f"Warning: Failed to create curve loop: {str(e)}")
        
        # if cannot be created
        for line in ordered_lines:
            line_usage_count[abs(line[0])] -= 1
        continue

Current curve loop lines: [1, 2, 3, 4, 5, 6, 7, 9, 8]
Sucessfully created 1
Current curve loop lines: [10, 15, 17, 18, 19, 20, 21, 22, 23, 25, 24, 16, 14, 13, 12, 11]
Sucessfully created 2
Current curve loop lines: [26, 31, 29, 28, 8, 27, 32, 33, 34, 35, 36, 37, 38, 30, 40, 39]
Sucessfully created 3


In [100]:
# store all the info of loops and relevant coor:
gmsh.model.occ.synchronize()
point_coords = {}
points = gmsh.model.getEntities(dim=0)

for dim, ptag in points:
    x, y, z = gmsh.model.getValue(dim, ptag, [])
    point_coords[ptag] = (x, y, z)


print(point_coords)
    

{1: (np.float64(13.763574600219727), np.float64(-17.313051223754883), np.float64(0.0)), 2: (np.float64(-2.9755916595458984), np.float64(32.077449798583984), np.float64(0.0)), 3: (np.float64(11.393732070922852), np.float64(-12.2442045211792), np.float64(0.0)), 4: (np.float64(17.68364906311035), np.float64(-15.6295804977417), np.float64(0.0)), 5: (np.float64(-25.497045516967773), np.float64(18.028989791870117), np.float64(0.0)), 6: (np.float64(-23.41815757751465), np.float64(14.5259370803833), np.float64(0.0)), 7: (np.float64(-41.280765533447266), np.float64(3.32720685005188), np.float64(0.0)), 8: (np.float64(-9.535467147827148), np.float64(-21.482484817504883), np.float64(0.0)), 9: (np.float64(6.313470840454102), np.float64(-14.5100736618042), np.float64(0.0)), 10: (np.float64(-5.114324569702148), np.float64(24.341978073120117), np.float64(0.0)), 11: (np.float64(2.9647464752197266), np.float64(6.405087947845459), np.float64(0.0)), 12: (np.float64(-2.1241512298583984), np.float64(4.13934

In [101]:
# Simple example to save as csv
import csv
import pandas as pd

records = []
loop_counter = 0
for curve_loop_tag, ordered_lines in zip(curve_loops, [ordered_lines]):
    loop_counter += 1
    point_order = 0
    
    for signed_line_tag, p_start, p_end in ordered_lines:
        point_order += 1
        
        # record start point of each line
        x, y, z = point_coords[p_start]
        records.append([
            curve_loop_tag, # loop idx
            point_order,
            signed_line_tag, # line idx
            p_start, # point idx
            x, y, z # coor
        ])
        
csv_file = "loop_info.csv"

with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "loop_tag",
        "order_in_loop",
        "line_tag",
        "point_tag",
        "x", "y", "z"
    ])
    writer.writerows(records)
  
df = pd.read_csv(csv_file)
df.head()


,loop_tag,order_in_loop,line_tag,point_tag,x,y,z
0,1,1,26,26,-0.757299,-16.925234,0.0
1,1,2,31,27,1.447321,-15.943912,0.0
2,1,3,29,30,-0.678137,-11.213931,0.0
3,1,4,28,29,3.905390,-9.171084,0.0
4,1,5,8,9,6.313471,-14.510074,0.0


In [102]:
# Visualize

plane_surfaces = []
for cl_tag in curve_loops:
    plane_surface_tag = gmsh.model.occ.addPlaneSurface([cl_tag])
    plane_surfaces.append(plane_surface_tag)
        
gmsh.model.occ.removeAllDuplicates()  # Boolean operation
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(1)


if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()